<a href="https://colab.research.google.com/github/StefanOVFDE/Academypublic/blob/main/Lesson_20201029.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#example

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
#create a kaggle folder
!mkdir ~/.kaggle
# Go on kaggle > Account > Create New API token
# Save the json file in your laptop in a dedicated folder
# copy the kaggle.json to folder created
!cp kaggle.json ~/.kaggle
#permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json
# Datasets available here: 
# https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009
!kaggle datasets download -d uciml/red-wine-quality-cortez-et-al-2009
!unzip red-wine-quality-cortez-et-al-2009.zip

Saving kaggle.json to kaggle.json
  0% 0.00/25.6k [00:00<?, ?B/s]
100% 25.6k/25.6k [00:00<00:00, 9.35MB/s]
Archive:  red-wine-quality-cortez-et-al-2009.zip
  inflating: winequality-red.csv     


In [ ]:
# Data types
# Most used: list

cities = ['Paris','Zurich','Milan','Berlin','Londn']

new_list = [1,2,3,4]

valueToBeRemoved = "4"
cities=[element for element in cities if element !=valueToBeRemoved]

val = 1
if val > 0:
  print('the value is positive')
elif val <0:
  print('the value is negaitve')
else:
  print('the value equals 0')


In [ ]:

fruits = ['apple', 'pear','cherry']
for stefan in fruits:
  print(stefan)
  


apple
pear
cherry


In [ ]:
def multiply(a,b):
  calc = a * b
  return calc

multiply(10,5)


50

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/winequality-red.csv')


In [ ]:
df[['fixed acidity','volatile acidity']].head(2)


,fixed acidity,volatile acidity
0,7.4,0.70
1,7.8,0.88


In [ ]:
mask1 = df['fixed acidity']>7
df[mask1]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1576,8.0,0.30,0.63,1.6,0.081,16.0,29.0,0.99588,3.30,0.78,10.8,6
1580,7.4,0.35,0.33,2.4,0.068,9.0,26.0,0.99470,3.36,0.60,11.9,6
1585,7.2,0.39,0.44,2.6,0.066,22.0,48.0,0.99494,3.30,0.84,11.5,6
1586,7.5,0.31,0.41,2.4,0.065,34.0,60.0,0.99492,3.34,0.85,11.4,6
